In [9]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize
from keras.models import Sequential, Model, model_from_json
import tensorflow as tf


In [15]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
#     print('Load file {} ...'.format(filename))
#     ds = pydicom.dcmread(filename)       
#     img = ds.pixel_array
    dcm = pydicom.dcmread(filename)
    
    if dcm.Modality=='DX' and dcm.BodyPartExamined=='CHEST' and dcm.PatientPosition in ['PA','AP']:
        img = dcm.pixel_array
        return img
    else:
        print("The input file is not a valid dicom file.")
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    proc_img = (img - img_mean) / img_std
    proc_img = ImageDataGenertor(proc_img, img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    with open(model_path, "r") as json_file:
        arch = json_file.read()
    
    model = model_from_json(arch)
    model.load_weights(weight_path)
        
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo 
    pred = model(img)
    if pred>thresh:
        return 'Positive'
    else:
        return 'Negative'
    

In [17]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "pneumonia_classifier_arch.json"
weight_path = "pneumonia_classifier.best.hdf5"

# This might be different if you did not use vgg16
IMG_SIZE=(1, 224,224,3) 

# loads the mean image value they used during training preprocessing
img_mean = 0 

# loads the std dev image value they used during training preprocessing
img_std = 1 

# loads model
my_model = load_model(model_path, weight_path)

#loads the threshold they chose for model classification 
thresh = 0.25

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model, img_proc,thresh)
    
    plt.imshow(img, cmap='gray')
    plt.show()
    print('Pred:' + pred + ' & Actual: ' + pydicom.dcmread(i).StudyDescription)

ValueError: Layer sequential_1 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[[137.00091472, 137.00091472, 137.00091472],
         [137.00091472, 137.00091472, 137.00091472],
         [137.00091472, 137.00091472, 137.00091472],
         ...,
         [137.00091472, 137.00091472, 137.00091472],
         [137.00091472, 137.00091472, 137.00091472],
         [137.00091472, 137.00091472, 137.00091472]],

        [[136.6213177 , 136.6213177 , 136.6213177 ],
         [136.6213177 , 136.6213177 , 136.6213177 ],
         [136.6213177 , 136.6213177 , 136.6213177 ],
         ...,
         [136.6213177 , 136.6213177 , 136.6213177 ],
         [136.6213177 , 136.6213177 , 136.6213177 ],
         [136.6213177 , 136.6213177 , 136.6213177 ]],

        [[140.59893089, 140.59893089, 140.59893089],
         [140.59893089, 140.59893089, 140.59893089],
         [140.59893089, 140.59893089, 140.59893089],
         ...,
         [140.598930